# Structure gemini outputs in a robust way

In [1]:
from google import genai


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """
)
print(response.text)

Oh, get ready for some truly soul-warming, flavor-packed Asian soups! Here are a few to tantalize your taste buds:

---

### 1. Vietnamese Pho Bo (Beef Noodle Soup)

**Yummy Description:** Imagine a giant, comforting hug in a bowl. This aromatic Vietnamese masterpiece features a deeply savory, crystal-clear broth simmered for hours with charred onions, ginger, and a fragrant spice medley of star anise, cinnamon, and cloves. It's brimming with tender slices of beef, slippery rice noodles, and a vibrant scattering of fresh herbs like cilantro and basil, crunchy bean sprouts, and a squeeze of lime. Each spoonful is a symphony of sweet, savory, and herbaceous notes, leaving you utterly satisfied and refreshed.

**Key Ingredients:**
*   **For the Broth:** Beef bones (marrow and knuckle), beef flank steak or brisket, yellow onions, ginger, star anise, cinnamon sticks, black cardamom pods, cloves, fish sauce, rock sugar, salt.
*   **Noodles:** Flat rice noodles (Bánh Phở).
*   **Beef for Serv

In [2]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients. Structure it in json format
    """
)
print(response.text)

```json
{
  "asian_soups": [
    {
      "name": "Pho Bo (Vietnamese Beef Noodle Soup)",
      "description": "A comforting and aromatic Vietnamese beef noodle soup, celebrated for its deeply savory broth simmered for hours with warming spices like star anise and cinnamon, served with tender slices of beef, slippery rice noodles, and a vibrant array of fresh herbs.",
      "ingredients": [
        "Beef bones",
        "Beef flank or brisket",
        "Beef sirloin (thinly sliced for serving)",
        "Rice noodles (banh pho)",
        "Onions",
        "Ginger",
        "Star anise",
        "Cinnamon sticks",
        "Cloves",
        "Black cardamom pods",
        "Fish sauce",
        "Sugar",
        "Scallions",
        "Cilantro",
        "Thai basil",
        "Bean sprouts",
        "Lime wedges",
        "Jalapeños or serrano peppers"
      ]
    },
    {
      "name": "Tom Yum Goong (Thai Hot and Sour Shrimp Soup)",
      "description": "A vibrant and exhilarating Thai hot a

In [3]:
from pydantic import BaseModel
from google import genai


class Recipe(BaseModel):
    recipe_name: str
    description: str
    ingredients: list[str]


client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""
    List a few asian soups recipes, a yummy description
    and list the ingredients
    """,
    config={"response_mime_type": "application/json", "response_schema": list[Recipe]},
)

print(response.text)

[{"recipe_name":"Ramen Noodle Soup","description":"A comforting and rich Japanese noodle soup, typically featuring a savory broth, tender noodles, and an array of toppings for a delightful culinary experience.","ingredients":["Ramen noodles","Pork belly (chashu)","Soft-boiled eggs","Nori (seaweed)","Scallions","Miso paste","Soy sauce","Chicken or pork broth","Garlic","Ginger"]},{"recipe_name":"Tom Yum Goong","description":"A spicy and sour Thai soup, famous for its vibrant flavors, fragrant herbs, and plump shrimp. It's a truly invigorating and aromatic dish.","ingredients":["Shrimp","Lemongrass","Galangal","Kaffir lime leaves","Mushrooms","Fish sauce","Lime juice","Chili paste (nam prik pao)","Cilantro","Tomatoes"]},{"recipe_name":"Pho Bo (Vietnamese Beef Noodle Soup)","description":"A beloved Vietnamese beef noodle soup, known for its deeply aromatic and clear broth, tender slices of beef, and a medley of fresh herbs and rice noodles.","ingredients":["Beef bones","Beef brisket or rar

In [4]:
recipes = response.parsed
type(recipes), len(recipes)

(list, 3)

In [5]:
recipes[0]

Recipe(recipe_name='Ramen Noodle Soup', description='A comforting and rich Japanese noodle soup, typically featuring a savory broth, tender noodles, and an array of toppings for a delightful culinary experience.', ingredients=['Ramen noodles', 'Pork belly (chashu)', 'Soft-boiled eggs', 'Nori (seaweed)', 'Scallions', 'Miso paste', 'Soy sauce', 'Chicken or pork broth', 'Garlic', 'Ginger'])

In [6]:
recipes[-1].recipe_name, recipes[-1].ingredients

('Pho Bo (Vietnamese Beef Noodle Soup)',
 ['Beef bones',
  'Beef brisket or rare steak',
  'Rice noodles',
  'Onions',
  'Ginger',
  'Star anise',
  'Cinnamon sticks',
  'Cardamom pods',
  'Fish sauce',
  'Bean sprouts',
  'Basil',
  'Lime',
  'Jalapeños'])

# Simulate house prices

In [7]:
from typing import Literal

class Home(BaseModel):
    price: int 
    monthly_fee: int 
    living_area: float 
    number_rooms: float
    address: str
    type: Literal["apartment", "house"]

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""List 20 apartments and houses in Sweden with their monthly_fee, their price, living area, 
    number of rooms, address, type if it is apartment or house. 
    All currencies are in SEK.""",
    config={"response_mime_type": "application/json", "response_schema": list[Home]},
)

homes = response.parsed
homes[:3]

[Home(price=3500000, monthly_fee=2500, living_area=75.5, number_rooms=3.0, address='Västra Hamngatan 10, Göteborg', type='apartment'),
 Home(price=6800000, monthly_fee=5000, living_area=140.2, number_rooms=5.0, address='Djurgårdsvägen 15, Stockholm', type='house'),
 Home(price=2100000, monthly_fee=3200, living_area=50.0, number_rooms=2.0, address='Södra Förstadsgatan 22, Malmö', type='apartment')]

In [8]:
import pandas as pd 

df = pd.DataFrame([dict(home) for home in homes])
df.head()

,price,monthly_fee,living_area,number_rooms,address,type
0,3500000,2500,75.5,3.0,"Västra Hamngatan 10, Göteborg",apartment
1,6800000,5000,140.2,5.0,"Djurgårdsvägen 15, Stockholm",house
2,2100000,3200,50.0,2.0,"Södra Förstadsgatan 22, Malmö",apartment
3,4900000,4000,110.0,4.0,"Slottsgatan 8, Uppsala",house
4,1850000,2800,40.5,1.0,"Storgatan 3, Lund",apartment


In [9]:
df.query("price < 2_000_000")

,price,monthly_fee,living_area,number_rooms,address,type
4,1850000,2800,40.5,1.0,"Storgatan 3, Lund",apartment
13,1950000,2700,45.0,1.5,"Lilla Fiskaregatan 9, Lund",apartment
